# Importación de bibliotecas

In [1]:
import os
from os import listdir
from os.path import isfile,isdir, join


from glob import glob
import numpy as np
import pandas as pd
import tensorflow as tf
#import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import keras
from keras.preprocessing import image
from keras import layers, models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D, Dropout,Activation,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD

#pip install sklearn
#pip install scikit-image
from sklearn.model_selection import train_test_split
#from skimage import color, io
#from scipy.misc import imresize  ## deprecated
from PIL import Image

import datetime

Using TensorFlow backend.


# Carga de imágenes

In [2]:
#Tamaño de la imagen
ih, iw = 120, 120
#Forma de la imagen: alto, ancho y número de canales
input_shape = (ih, iw, 3)


#Número de clases
num_class = 12
#Cuantas veces entrenar
epochs = 30


# Batch para hacer cada entrenamiento.
# Lee 50 'batch_size' imagenes antes de actualizar los parametros.
# Las carga a memoria
batch_size = 50 #1


# Número de imagenes en train
num_train = 6000 #18632
#Número de imagenes en test
num_test = 750


epoch_steps = num_train // batch_size
test_steps = num_test // batch_size


""" Imágenes de entrenamiento """
train_f = pd.read_csv('train.csv')

gentrain = ImageDataGenerator(rescale = 1. / 255)

train_ds = gentrain.flow_from_dataframe(train_f,directory = 'minitrain_images',
                                             x_col = 'image',
                                             y_col = 'labels',
                                             batch_size=batch_size,
                                             target_size=(iw, ih),
                                             class_mode = 'categorical')

print(train_ds.class_indices)

""" Imágenes de prueba """

test_f = pd.read_csv('train.csv')

gentest = ImageDataGenerator(rescale=1. / 255)

test_ds = gentest.flow_from_dataframe(test_f,directory = 'minitest_images',
                                             x_col = 'image',
                                             y_col = 'labels',
                                             batch_size=batch_size,
                                             target_size=(iw, ih),
                                             class_mode = 'categorical')

print(test_ds.class_indices)

#para cargar pesos de la red desde donde se quedó la ultima vez
#filename = "cvsd.h5"
#model.load_weights(filename)  #comentar si se comienza desde cero.
###



C:\Users\artur\anaconda3\envs\negative_creep\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:282: UserWarning: Found 12632 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 6000 validated image filenames belonging to 12 classes.
{'complex': 0, 'frog_eye_leaf_spot': 1, 'frog_eye_leaf_spot complex': 2, 'healthy': 3, 'powdery_mildew': 4, 'powdery_mildew complex': 5, 'rust': 6, 'rust complex': 7, 'rust frog_eye_leaf_spot': 8, 'scab': 9, 'scab frog_eye_leaf_spot': 10, 'scab frog_eye_leaf_spot complex': 11}
Found 750 validated image filenames belonging to 12 classes.
{'complex': 0, 'frog_eye_leaf_spot': 1, 'frog_eye_leaf_spot complex': 2, 'healthy': 3, 'powdery_mildew': 4, 'powdery_mildew complex': 5, 'rust': 6, 'rust complex': 7, 'rust frog_eye_leaf_spot': 8, 'scab': 9, 'scab frog_eye_leaf_spot': 10, 'scab frog_eye_leaf_spot complex': 11}


C:\Users\artur\anaconda3\envs\negative_creep\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:282: UserWarning: Found 17882 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


# Modelo y entrenamiento

In [3]:
model = Sequential()
model.add(Conv2D(10, (5, 5), input_shape=(ih, iw,3))) #50, 50
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(10, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(20, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(12))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tbCallBack = keras.callbacks.TensorBoard(log_dir=log_dir,
                                         histogram_freq=1, 
                                         write_graph=True, 
                                         write_images=True)
#python -m tensorboard.main --logdir=/Graph  <- Para correr Tensor board
#tensorboard  --logdir Graph/
print('Logs:')
print(log_dir)
print('______________________________________________________________')


history = model.fit_generator(
                train_ds,
                steps_per_epoch=epoch_steps,
                epochs=epochs,
                validation_data=test_ds,
                validation_steps=test_steps,
                callbacks=[tbCallBack]
                )

Logs:
logs/fit/20220510-000000
______________________________________________________________
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
  1/120 [..............................] - ETA: 0s - loss: 2.4588 - accuracy: 0.1200WARNING:tensorflow:From C:\Users\artur\anaconda3\envs\negative_creep\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
120/120 [==============================] - 3234s 27s/step - loss: 1.9875 - accuracy: 0.2500 - val_loss: 1.7123 - val_accuracy: 0.4280
Epoch 2/30
120/120 [==============================] - 2944s 25s/step - loss: 1.7924 - accuracy: 0.3285 - val_loss: 1.7121 - val_accuracy: 0.4067
Epoch 3/30
120/120 [==============================] - 2952s 25s/step - loss: 1.7016 - accuracy: 0.3802 - val_loss: 1.5952 - val_accuracy: 0.3907
Epoch 4/3

KeyboardInterrupt: 

## Guardando los pesos

In [4]:
model.save_weights('Conv2D_weights_nofitting2.h5')

# Resumen del modelo y gráficas

In [5]:
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.savefig('Conv2D_accuracy.svg')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#plt.savefig('Conv2D_loss.svg')
plt.show()

NameError: name 'history' is not defined